# Generate daily prices using daily data service - F#

### Overview
Daily prices is designed to illustrate price movements for an instrument over a look back period.

This sample demonstrates how to directly request and plot daily prices.<br>

It enables to retrieve **on-demand** daily prices data by calling a dedicated service on a **daily** data store. 

### Inputs/outputs
Daily prices sample requires instrument's identifier as per input. It returns daily prices and volumes.
This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

### Services used
This sample uses *gRPC requests* in order to retrieve daily bars from the dedicated hosted service. The queried endpoint in this script are:
* *DailyPricesService*: to directly retrieve daily price objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Daily.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run daily prices sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.36.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Daily.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *daily prices* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Service called: *DailyPricesService* in order to retrieve daily data

In [ ]:
// Set instrument
let ticker = "AAPL"
let exchange = "XNGS"

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
// Instantiate the daily prices service
let service = new DailyPricesService.DailyPricesServiceClient(channel)

// Create and process prices data request
let request = 
    new DailyPricesRequest(
        Identifier = new Identifier (Exchange = exchange, Ticker = ticker)
    )

let reply = service.DailyPrices(request, headers)

In [ ]:
//Visualize request results
reply.Data
|> display

### Step 4: Visualize data

#### 4.1 Plot the prices and volumes
The following code snippets retrive *daily prices* over the look back period from the request result and plot them with the package of your choice.<br><i>`XPlot.Plotly`</i> is used as per open source display package.

In [ ]:
// Extract the daily prices and the volumes
let prices = 
    reply.Data
    |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Price))
let volumes = 
    reply.Data
    |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Volume))

In [ ]:
// Create the graph
let pricesAndVolumes = 
    [
        new Scattergl(
            name = "Price",
            mode = "lines",
            x = (prices |> Seq.map fst),
            y = (prices |> Seq.map snd)
        )
        new Scattergl(
            name = "Volume",
            mode = "bar",
            x = (volumes |> Seq.map fst),
            y = (volumes |> Seq.map snd),
            yaxis = "y2"
        )
    ]


let layout = 
    new Layout.Layout(
        title = $"Daily price & Volume for {ticker}",
        yaxis = new Yaxis(
            title = "Price", 
            autorange = true,
            side = "left"
        ),
        yaxis2 = new Yaxis(
            title = "Volume", 
            autorange = true,
            side = "right"
        )
    )

Chart.Plot(pricesAndVolumes, layout)
|> Chart.WithXTitle("Time")
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display